In [40]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

1.2.2


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import pyodbc

In [42]:
serverName = "sqlserver-pw.database.windows.net" 
databaseName = "db-ai"
username = "girls"
password = "Password?"

In [43]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+serverName+';DATABASE='+databaseName+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [44]:
query = "SELECT TOP 10000 * FROM try.DatiConsumi_ML WHERE [Soc.Vend.Prec.] is not null"

#Load the dataset into a pandas DataFrame
df = pd.read_sql(query, cnxn)


C:\Users\IreneDeNevi\AppData\Local\Temp\ipykernel_25540\4023592982.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


In [45]:
df.isnull().sum()

Canale                 0
CodAgenzia             0
ClusterAgenzia         0
CodContratto           0
CategoriaCespite       0
                    ... 
Mese60              9933
MesiBEP                0
MesiAttivi             0
UltimoReddito          0
BEP                    0
Length: 98, dtype: int64

In [46]:
print(df.head(20))

   Canale CodAgenzia ClusterAgenzia CodContratto CategoriaCespite Cespite  \
0       6          1              1       203252                5       4   
1       6          1              1       230585                5       4   
2       6          1              1       230586                5       4   
3       6          1              1       230588                5       4   
4       6          1              1       230591                5       4   
5       6          1              1       230592                5       4   
6       6          1              1       230603                5       4   
7       6          1              1       230605                5       4   
8       6          1              1       230611                5       4   
9       6          1              1       230618                5       4   
10      6          1              1       230633                5       4   
11      6          1              1       230636                5       4   

In [47]:
df.columns

Index(['Canale', 'CodAgenzia', 'ClusterAgenzia', 'CodContratto',
       'CategoriaCespite', 'Cespite', 'TipologiaUtenza', 'DataTrasf.entr.',
       'DataTrasf.usc.', 'MotivoBloccoCalc.', 'Anagrafica',
       'SettoreMerceologico', 'ListinoProposto', 'Soc.Vend.Prec.',
       'ClusterSocVend', 'Localita', 'Cod.ISTAT', 'Provincia',
       'RegioneItaliana', 'Zona', 'ModoPagamento', 'Permanenza',
       'ConsumoMedio_annuo', 'ConsumoTotale', 'MargineFissoQF_annuo',
       'MargineFissoTotale', 'MargineVariabileQV', 'MargineVariabileQV_annuo',
       'MargineVariabileTotale', 'MargineTotale', 'Provvigioni',
       'Acquisizione', 'CostoAnnuo_Post-vendita_Customer-care',
       'RedditoTotale', 'Mese01', 'Mese02', 'Mese03', 'Mese04', 'Mese05',
       'Mese06', 'Mese07', 'Mese08', 'Mese09', 'Mese10', 'Mese11', 'Mese12',
       'Mese13', 'Mese14', 'Mese15', 'Mese16', 'Mese17', 'Mese18', 'Mese19',
       'Mese20', 'Mese21', 'Mese22', 'Mese23', 'Mese24', 'Mese25', 'Mese26',
       'Mese27', 'Mes

In [48]:
df.drop(['Mese01', 'Mese02', 'Mese03', 'Mese04', 'Mese05',
       'Mese06', 'Mese07', 'Mese08', 'Mese09', 'Mese10', 'Mese11', 'Mese12',
       'Mese13', 'Mese14', 'Mese15', 'Mese16', 'Mese17', 'Mese18', 'Mese19',
       'Mese20', 'Mese21', 'Mese22', 'Mese23', 'Mese24', 'Mese25', 'Mese26',
       'Mese27', 'Mese28', 'Mese29', 'Mese30', 'Mese31', 'Mese32', 'Mese33',
       'Mese34', 'Mese35', 'Mese36', 'Mese37', 'Mese38', 'Mese39', 'Mese40',
       'Mese41', 'Mese42', 'Mese43', 'Mese44', 'Mese45', 'Mese46', 'Mese47',
       'Mese48', 'Mese49', 'Mese50', 'Mese51', 'Mese52', 'Mese53', 'Mese54',
       'Mese55', 'Mese56', 'Mese57', 'Mese58', 'Mese59', 'Mese60', 'DataTrasf.entr.',
       'DataTrasf.usc.'], axis=1,inplace=True)



In [49]:
df.isnull().sum()

Canale                                   0
CodAgenzia                               0
ClusterAgenzia                           0
CodContratto                             0
CategoriaCespite                         0
Cespite                                  0
TipologiaUtenza                          0
MotivoBloccoCalc.                        0
Anagrafica                               0
SettoreMerceologico                      0
ListinoProposto                          0
Soc.Vend.Prec.                           0
ClusterSocVend                           0
Localita                                 0
Cod.ISTAT                                0
Provincia                                0
RegioneItaliana                          0
Zona                                     0
ModoPagamento                            0
Permanenza                               0
ConsumoMedio_annuo                       0
ConsumoTotale                            0
MargineFissoQF_annuo                     0
MargineFiss

In [50]:
from sklearn.datasets import make_regression
# create datasets
y = df[['MesiAttivi', 'RedditoTotale', 'BEP']]                                         #Targets
X = df[[i for i in list(df.columns) if i != 'RedditoTotale' and 'MesiAttivi' and 'BEP']] #features
# summarize dataset
print(X.shape, y.shape)

(10000, 35) (10000, 3)


In [51]:
X.isnull().sum()

Canale                                   0
CodAgenzia                               0
ClusterAgenzia                           0
CodContratto                             0
CategoriaCespite                         0
Cespite                                  0
TipologiaUtenza                          0
MotivoBloccoCalc.                        0
Anagrafica                               0
SettoreMerceologico                      0
ListinoProposto                          0
Soc.Vend.Prec.                           0
ClusterSocVend                           0
Localita                                 0
Cod.ISTAT                                0
Provincia                                0
RegioneItaliana                          0
Zona                                     0
ModoPagamento                            0
Permanenza                               0
ConsumoMedio_annuo                       0
ConsumoTotale                            0
MargineFissoQF_annuo                     0
MargineFiss

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)

In [53]:
from sklearn.linear_model import LinearRegression

In [54]:
model = LinearRegression()

In [55]:
model.fit(X_train, y_train)

LinearRegression()

In [56]:
y_pred = model.predict(X_test) #mesi attivi, reddito, bep
print(y_pred[0])

[ 1.90000000e+01 -4.52858252e+01  1.30968996e-15]
